Notebook with functions for reading in data and getting various ensemble means for all experiments (0.1, 0.2, and 0.5 TW; surf, therm, mid, and bot; const, 2xCO2, and 4xCO2). This is designed to be used with the notebook read_and_calculate.ipynb.

Currently only have the function for constant CO2 experiments.

In [39]:
import numpy as np
import xarray as xr

# modules for using datetime variables
import datetime
from datetime import time

import warnings
warnings.filterwarnings('ignore')

import cftime
from pandas.errors import OutOfBoundsDatetime  # Import the specific error

from xclim import ensembles

In [2]:
myVars = globals()

In [38]:
# can delete this cell once I'm done debugging
%run /home/Kiera.Lowman/Kd-sensitivity-analysis/notebooks/read_and_calculate.ipynb # to get selecting_basins function
%run /home/Kiera.Lowman/Kd-sensitivity-analysis/notebooks/plotting_functions.ipynb
%run /home/Kiera.Lowman/Kd-sensitivity-analysis/notebooks/read_mem1_functions.ipynb
%run /home/Kiera.Lowman/Kd-sensitivity-analysis/notebooks/custom_moc_functions.ipynb

# Function for ensemble means

In [4]:
const_exp_root = '_1860IC_200yr_'

# exp root for 2xCO2 exps is different for mem1 compared to mem2 and mem3
mem1_doub_exp_root = '_2xCO2_1860IC_200yr_'
mem2_3_doub_exp_root = '_2xCO2_200yr_'

quad_exp_root = '_4xCO2_51-201_'

profiles = ['surf','therm','mid','bot'] #'uni',
power_inputs = ['0.1TW', '0.2TW', '0.5TW'] #, '1TW'
power_var_suff = ['0p1TW', '0p2TW', '0p5TW'] #,'1TW'

# prof_strings = ["Surface-Enhanced","Thermocline-Enhanced","Middepth-Enhanced","Bottom-Enhanced"] #"Uniformly-Enhanced",
prof_strings = ["Surf","Therm","Mid","Bot"] #"Uniformly-Enhanced",

In [34]:
def get_const_CO2_ens_dat(avg_period,mem1_start,mem1_end,var_list,verbose=False):

    """
    Returns variables containing the ensemble-mean raw data and variables containing the ensemble-mean anomaly data. Anomalies are
    calculated as the difference relative to the control run during the period corresponding to an ensemble member (i.e. the anomalies
    for ensemble member 2 for year 201 to 400 are taking as the difference relative to year 201 to 400 of the control run).

        Args:
            avg_period (int): number of years for av/ts period
            mem1_start (int)
            mem1_end (int)
            var_list (str list): list of variables to read (e.g. var_list = ["temp", "N2", "age", "rhopot2", "salt"])
        Returns:
            has no return variables, but creates xarray datasets by using myVars = globals()
            
    """

    mem2_start = mem1_start + 200
    mem2_end = mem1_end + 200

    mem3_start = mem1_start + 400
    mem3_end = mem1_end + 400

    # control
    ctrl_mem1 = get_pp_av_data('tune_ctrl_1860IC_200yr',mem1_start,mem1_end,avg_period,pp_type='av-annual')
    ctrl_mem2 = get_pp_av_data('tune_ctrl_1860IC_201-2001',mem2_start,mem2_end,avg_period,pp_type='av-annual',time_decoding=False)
    ctrl_mem3 = get_pp_av_data('tune_ctrl_1860IC_201-2001',mem3_start,mem3_end,avg_period,pp_type='av-annual',time_decoding=False)

    for idx, time_val in enumerate(ctrl_mem1.time.values):
        ctrl_mem2.time.values[idx] = time_val
        ctrl_mem3.time.values[idx] = time_val

    raw_ctrl = ensembles.create_ensemble([ctrl_mem1,ctrl_mem2,ctrl_mem3]).mean("realization")
    raw_ctrl = raw_ctrl[var_list]
    raw_ctrl_name = f"const_ctrl_{mem1_start}_{mem1_end}"
    myVars.__setitem__(raw_ctrl_name, raw_ctrl)

    if verbose:
        print(f"{raw_ctrl_name} done")
    
    for prof in profiles:
        for idx, power_str in enumerate(power_inputs):

            mem1_exp_name = prof+const_exp_root+power_str
            mem1_ds = get_pp_av_data(mem1_exp_name,mem1_start,mem1_end,avg_period,pp_type='av-annual')

            if mem1_start == 1 and mem1_end == 25:
                if prof == "therm" and power_str == "0.5TW":
                    continue #mem3 pp missing
                elif prof == "mid":
                    continue #mem3 pp missing
                elif prof == "bot":
                    continue

            mem2_exp_name = "mem2_"+prof+const_exp_root+power_str
            mem2_ds = get_pp_av_data(mem2_exp_name,mem2_start,mem2_end,avg_period,pp_type='av-annual',time_decoding=False)

            mem3_exp_name = "mem3_"+prof+const_exp_root+power_str
            mem3_ds = get_pp_av_data(mem3_exp_name,mem3_start,mem3_end,avg_period,pp_type='av-annual',time_decoding=False)

            for i, time_val in enumerate(mem1_ds.time.values):
                mem2_ds.time.values[i] = time_val
                mem3_ds.time.values[i] = time_val
                
            raw_ens = ensembles.create_ensemble([mem1_ds,mem2_ds,mem3_ds]).mean("realization")
            raw_ens = raw_ens[var_list]
            raw_ens_name = f"{prof}_{power_var_suff[idx]}_{mem1_start}_{mem1_end}"
            myVars.__setitem__(raw_ens_name, raw_ens)

            mem1_diff = mem1_ds[var_list] - ctrl_mem1[var_list]
            mem2_diff = mem2_ds[var_list] - ctrl_mem2[var_list]
            mem3_diff = mem3_ds[var_list] - ctrl_mem3[var_list]

            diff_ens = ensembles.create_ensemble([mem1_diff,mem2_diff,mem3_diff]).mean("realization")
            diff_ens = diff_ens[var_list]
            diff_ens['wet'] = ctrl_mem1['wet']
            diff_ens['dxt'] = ctrl_mem1['dxt']
            diff_ens['dyt'] = ctrl_mem1['dyt']
            diff_ens['areacello'] = ctrl_mem1['areacello']

            # alternative to get more stats is:
            # diff_ens = ensembles.ensemble_mean_std_max_min(diff_ens)
            # for var in var_list:
            #     mean_var_name = var + '_mean'
            #     diff_ens = diff_ens.rename({mean_var_name: var})

            diff_ens_name = f"{raw_ens_name}_diff"
            myVars.__setitem__(diff_ens_name, diff_ens)
    
            horiz_avg_diff_name = f"{diff_ens_name}_mean"
            horiz_avg_diff = xr.Dataset()
            for var in var_list:
                horiz_avg_diff[var] = horizontal_mean(myVars[diff_ens_name][var],mem1_ds)
            myVars.__setitem__(horiz_avg_diff_name, horiz_avg_diff)

            if verbose:
                print(f'{raw_ens_name}, {diff_ens_name}, {horiz_avg_diff_name} done')

In [35]:
get_const_CO2_ens_dat(25,1,25,["temp", "N2", "age", "rhopot2", "salt"],verbose=True)

const_ctrl_1_25 done
surf_0p1TW_1_25, surf_0p1TW_1_25_diff, surf_0p1TW_1_25_diff_mean done
surf_0p2TW_1_25, surf_0p2TW_1_25_diff, surf_0p2TW_1_25_diff_mean done
surf_0p5TW_1_25, surf_0p5TW_1_25_diff, surf_0p5TW_1_25_diff_mean done
therm_0p1TW_1_25, therm_0p1TW_1_25_diff, therm_0p1TW_1_25_diff_mean done
therm_0p2TW_1_25, therm_0p2TW_1_25_diff, therm_0p2TW_1_25_diff_mean done


In [36]:
get_const_CO2_ens_dat(25,26,50,["temp", "N2", "age", "rhopot2", "salt"],verbose=True)

const_ctrl_26_50 done


FileNotFoundError: [Errno 2] No such file or directory: '/archive/Kiera.Lowman/FMS2019.01.03_devgfdl_20201120_kiera/mem3_surf_1860IC_200yr_0.1TW/gfdl.ncrc5-intel23-prod/pp/ocean_monthly_z/av/annual_25yr/ocean_monthly_z.0426-0450.ann.nc'

In [37]:
get_const_CO2_ens_dat(25,51,75,["temp", "N2", "age", "rhopot2", "salt"],verbose=True)

const_ctrl_51_75 done
surf_0p1TW_51_75, surf_0p1TW_51_75_diff, surf_0p1TW_51_75_diff_mean done
surf_0p2TW_51_75, surf_0p2TW_51_75_diff, surf_0p2TW_51_75_diff_mean done


FileNotFoundError: [Errno 2] No such file or directory: '/archive/Kiera.Lowman/FMS2019.01.03_devgfdl_20201120_kiera/mem3_surf_1860IC_200yr_0.5TW/gfdl.ncrc5-intel23-prod/pp/ocean_monthly_z/av/annual_25yr/ocean_monthly_z.0451-0475.ann.nc'

In [20]:
get_const_CO2_ens_dat(25,76,100,["temp", "N2", "age", "rhopot2", "salt"],verbose=True)

const_ctrl_76_100 done
surf_0p1TW_76_100, surf_0p1TW_76_100_diff, surf_0p1TW_76_100_diff_mean done
surf_0p2TW_76_100, surf_0p2TW_76_100_diff, surf_0p2TW_76_100_diff_mean done
surf_0p5TW_76_100, surf_0p5TW_76_100_diff, surf_0p5TW_76_100_diff_mean done
therm_0p1TW_76_100, therm_0p1TW_76_100_diff, therm_0p1TW_76_100_diff_mean done
therm_0p2TW_76_100, therm_0p2TW_76_100_diff, therm_0p2TW_76_100_diff_mean done
therm_0p5TW_76_100, therm_0p5TW_76_100_diff, therm_0p5TW_76_100_diff_mean done
mid_0p1TW_76_100, mid_0p1TW_76_100_diff, mid_0p1TW_76_100_diff_mean done
mid_0p2TW_76_100, mid_0p2TW_76_100_diff, mid_0p2TW_76_100_diff_mean done
mid_0p5TW_76_100, mid_0p5TW_76_100_diff, mid_0p5TW_76_100_diff_mean done
bot_0p1TW_76_100, bot_0p1TW_76_100_diff, bot_0p1TW_76_100_diff_mean done
bot_0p2TW_76_100, bot_0p2TW_76_100_diff, bot_0p2TW_76_100_diff_mean done
bot_0p5TW_76_100, bot_0p5TW_76_100_diff, bot_0p5TW_76_100_diff_mean done


In [19]:
get_const_CO2_ens_dat(25,176,200,["temp", "N2", "age", "rhopot2", "salt"],verbose=True)

const_ctrl_176_200 done
surf_0p1TW_176_200, surf_0p1TW_176_200_diff, surf_0p1TW_176_200_diff_mean done
surf_0p2TW_176_200, surf_0p2TW_176_200_diff, surf_0p2TW_176_200_diff_mean done
surf_0p5TW_176_200, surf_0p5TW_176_200_diff, surf_0p5TW_176_200_diff_mean done
therm_0p1TW_176_200, therm_0p1TW_176_200_diff, therm_0p1TW_176_200_diff_mean done
therm_0p2TW_176_200, therm_0p2TW_176_200_diff, therm_0p2TW_176_200_diff_mean done
therm_0p5TW_176_200, therm_0p5TW_176_200_diff, therm_0p5TW_176_200_diff_mean done
mid_0p1TW_176_200, mid_0p1TW_176_200_diff, mid_0p1TW_176_200_diff_mean done
mid_0p2TW_176_200, mid_0p2TW_176_200_diff, mid_0p2TW_176_200_diff_mean done
mid_0p5TW_176_200, mid_0p5TW_176_200_diff, mid_0p5TW_176_200_diff_mean done
bot_0p1TW_176_200, bot_0p1TW_176_200_diff, bot_0p1TW_176_200_diff_mean done
bot_0p2TW_176_200, bot_0p2TW_176_200_diff, bot_0p2TW_176_200_diff_mean done
bot_0p5TW_176_200, bot_0p5TW_176_200_diff, bot_0p5TW_176_200_diff_mean done
